## Scraping Data from CodeClimate


In [1]:
import requests
import json
import os
from dotenv import load_dotenv
import datetime

In [2]:
load_dotenv()

True

In [3]:
def iso_time_to_unix_epoch(iso_time: str):
    return int(datetime.datetime.strptime(iso_time, "%Y-%m-%dT%H:%M:%SZ").timestamp())

In [5]:
token = os.getenv("CODECLIMATE_ACCESS_TOKEN")
github_token = os.getenv("GITHUB_ACCESS_TOKEN")
demo_github_slug = "NotAlpha45"
demo_repository_name = "vant"
default_headers = {
    "Accept": "application/vnd.api+json",
    "Authorization": f"Token token={token}",
}
github_users_and_repos = {
    "youzan": "vant",
}

In [6]:
response = requests.get(
    url=f"https://api.codeclimate.com/v1/repos?github_slug={demo_github_slug}/{demo_repository_name}",
    headers=default_headers,
)

repository_data = response.json()["data"][0]
repository_data

{'id': '655a1112bfaf6e2b94a00f9d',
 'type': 'repos',
 'attributes': {'analysis_version': 77760,
  'badge_token': 'd35f358f74604fabea81',
  'branch': 'main',
  'created_at': '2023-11-19T13:43:46.719Z',
  'delegated_config_repo_id': '',
  'diff_coverage_enforced': True,
  'diff_coverage_threshold': 50,
  'enable_notifications': True,
  'github_slug': 'NotAlpha45/vant',
  'human_name': 'vant',
  'last_activity_at': '2023-11-25T15:56:25.336Z',
  'test_reporter_id': 'ec856ac497081c3044fd08f5c4f658602df56ecc90c632c5cb0dccb5beff9f04',
  'total_coverage_enforced': True,
  'vcs_database_id': '720749938',
  'vcs_host': 'https://github.com'},
 'relationships': {'latest_default_branch_snapshot': {'data': {'id': '656219147df70a000102d4dd',
    'type': 'snapshots'}},
  'latest_default_branch_test_report': {'data': None},
  'account': {'data': None}},
 'links': {'self': 'https://codeclimate.com/github/NotAlpha45/vant',
  'services': 'https://api.codeclimate.com/v1/repos/655a1112bfaf6e2b94a00f9d/servi

In [7]:
repository_id = repository_data["id"]
repository_id

'655a1112bfaf6e2b94a00f9d'

In [8]:
latest_snapshot_id = repository_data["relationships"]["latest_default_branch_snapshot"][
    "data"
]["id"]
latest_snapshot_id

'656219147df70a000102d4dd'

In [9]:
# Get time series metrics (remediation_minutes) of the repository

response = requests.get(
    url=f"https://api.codeclimate.com/v1/repos/{repository_id}/metrics/remediation_minutes?filter[from]=2023-08-20&filter[to]=2023-11-19",
    headers=default_headers,
)

response.json()["data"]

{'id': '655a1137a0367900010000b9',
 'type': 'metrics',
 'attributes': {'name': 'remediation_minutes',
  'points': [{'timestamp': 1691971200, 'value': None},
   {'timestamp': 1692576000, 'value': None},
   {'timestamp': 1693180800, 'value': None},
   {'timestamp': 1693785600, 'value': None},
   {'timestamp': 1694390400, 'value': None},
   {'timestamp': 1694995200, 'value': None},
   {'timestamp': 1695600000, 'value': None},
   {'timestamp': 1696204800, 'value': None},
   {'timestamp': 1696809600, 'value': None},
   {'timestamp': 1697414400, 'value': None},
   {'timestamp': 1698019200, 'value': None},
   {'timestamp': 1698624000, 'value': 58330.36},
   {'timestamp': 1699228800, 'value': 58330.36},
   {'timestamp': 1699833600, 'value': 58330.36}]}}

In [10]:
# Get the ref points of the repository

response = requests.get(
    url=f"https://api.codeclimate.com/v1/repos/{repository_id}/ref_points",
    headers=default_headers,
)

ref_points = response.json()["data"]
ref_points[:]

[{'id': '65621914d23cfe0001003cd6',
  'type': 'ref_points',
  'attributes': {'analyzed': True,
   'branch': 'main',
   'commit_sha': '62779129b46f4f3a13ec60139d5236a2fb0e68f9',
   'created_at': '2023-11-25T15:56:04.345Z',
   'ref': 'refs/heads/main'},
  'relationships': {'snapshot': {'data': {'id': '656219147df70a000102d4dd',
     'type': 'snapshots'}}}},
 {'id': '655a1122377b2200010063ff',
  'type': 'ref_points',
  'attributes': {'analyzed': True,
   'branch': 'main',
   'commit_sha': 'f325fc862c262d662f746aea30878007a64e97f7',
   'created_at': '2023-11-19T13:44:02.499Z',
   'ref': 'refs/heads/main'},
  'relationships': {'snapshot': {'data': {'id': '655a1122678a9d0001048d09',
     'type': 'snapshots'}}}},
 {'id': '655a11229ad2c02d5966d069',
  'type': 'ref_points',
  'attributes': {'analyzed': True,
   'branch': 'main',
   'commit_sha': 'f325fc862c262d662f746aea30878007a64e97f7',
   'created_at': '2023-11-19T05:11:29.000Z',
   'ref': 'refs/heads/main'},
  'relationships': {'snapshot': 

In [16]:
# Get issues of the repository

response = requests.get(
    url=f"https://api.codeclimate.com/v1/repos/{repository_id}/snapshots/{latest_snapshot_id}/issues",
    headers=default_headers,
)

issues = response.json()["data"]
issues[:2]

[{'id': '65621928e199f300010004a7',
  'type': 'issues',
  'attributes': {'categories': ['Duplication'],
   'check_name': 'identical-code',
   'constant_name': 'packages/vant-area-data/src/index.ts',
   'content': {'body': "## Duplicated Code\n\nDuplicated code can lead to software that is hard to understand and difficult to change. The Don't Repeat Yourself (DRY) principle states:\n\n> Every piece of knowledge must have a single, unambiguous, authoritative representation within a system.\n\nWhen you violate DRY, bugs and maintenance problems are sure to follow. Duplicated code has a tendency to both continue to replicate and also to diverge (leaving bugs as two similar implementations differ in subtle ways).\n\n## Tuning\n\n**This issue has a mass of 82**.\n\nWe set useful threshold defaults for the languages we support but you may want to adjust these settings based on your project guidelines.\n\nThe threshold configuration represents the minimum [mass](https://docs.codeclimate.com/do

## Github API Data Extraction


In [17]:
github_request_headers = {
    "Accept": "application/vnd.github.v3+json",
    "Authorization": f"token {github_token}",
}

In [18]:
response = requests.get(
    url=f"https://api.github.com/repos/{demo_github_slug}/{demo_repository_name}/commits",
    headers=github_request_headers,
)

commits = response.json()
commits
# Sort the commits by date
commits.sort(
    key=lambda commit: iso_time_to_unix_epoch(commit["commit"]["author"]["date"])
)
commits[:2]

[{'sha': '5829f9aa72d13b69852298c6b5eb2d8b46b48c67',
  'node_id': 'C_kwDOKvXFctoAKDU4MjlmOWFhNzJkMTNiNjk4NTIyOThjNmI1ZWIyZDhiNDZiNDhjNjc',
  'commit': {'author': {'name': 'neverland',
    'email': 'jait.chen@foxmail.com',
    'date': '2023-10-28T11:29:29Z'},
   'committer': {'name': 'GitHub',
    'email': 'noreply@github.com',
    'date': '2023-10-28T11:29:29Z'},
   'message': 'docs(Tabs): add show-header prop (#12408)',
   'tree': {'sha': '41a5c0ad57ee8ac869b619ee8e4bd178a5f369ad',
    'url': 'https://api.github.com/repos/NotAlpha45/vant/git/trees/41a5c0ad57ee8ac869b619ee8e4bd178a5f369ad'},
   'url': 'https://api.github.com/repos/NotAlpha45/vant/git/commits/5829f9aa72d13b69852298c6b5eb2d8b46b48c67',
   'comment_count': 0,
   'verification': {'verified': True,
    'reason': 'valid',
    'signature': '-----BEGIN PGP SIGNATURE-----\n\nwsBcBAABCAAQBQJlPPCZCRBK7hj4Ov3rIwAAGegIADtPfnaMbH/T7tnsUOrq/QW1\neYLM88kJWGBsKJS50DxO57ceMQiUQOfHuhv6ZQohoiEZ23IXboee5XVGRvlrO13d\nUhqFL4R1xk66nfeK1fzZX54

In [19]:
# Get the releases for the repository

response = requests.get(
    url=f"https://api.github.com/repos/youzan/vant/releases",
    headers=github_request_headers,
)

releases = list(response.json())

# Sort the releases by published date
releases.sort(key=lambda release: iso_time_to_unix_epoch(release["published_at"]))

# Take the url, tag_name, target_commitsh, id and published_at fields
releases = [
    {
        "url": release["url"],
        "tag_name": release["tag_name"],
        "target_commitish": release["target_commitish"],
        "id": release["id"],
        "published_at": release["published_at"],
    }
    for release in releases
]

tag_names_in_releases = [release["tag_name"] for release in releases]
release_timestamps = [release["published_at"] for release in releases]

tag_names_in_releases.sort()

releases, tag_names_in_releases

([{'url': 'https://api.github.com/repos/youzan/vant/releases/94548765',
   'tag_name': 'v4.1.0',
   'target_commitish': 'main',
   'id': 94548765,
   'published_at': '2023-03-05T11:47:24Z'},
  {'url': 'https://api.github.com/repos/youzan/vant/releases/96106954',
   'tag_name': 'v4.1.1',
   'target_commitish': 'main',
   'id': 96106954,
   'published_at': '2023-03-19T12:44:26Z'},
  {'url': 'https://api.github.com/repos/youzan/vant/releases/96922877',
   'tag_name': 'v4.1.2',
   'target_commitish': 'main',
   'id': 96922877,
   'published_at': '2023-03-26T04:13:32Z'},
  {'url': 'https://api.github.com/repos/youzan/vant/releases/99645904',
   'tag_name': 'v4.2.0',
   'target_commitish': 'main',
   'id': 99645904,
   'published_at': '2023-04-16T03:14:32Z'},
  {'url': 'https://api.github.com/repos/youzan/vant/releases/101405585',
   'tag_name': 'v4.2.1',
   'target_commitish': 'main',
   'id': 101405585,
   'published_at': '2023-04-30T13:17:33Z'},
  {'url': 'https://api.github.com/repos/you

In [20]:
release_timestamps

['2023-03-05T11:47:24Z',
 '2023-03-19T12:44:26Z',
 '2023-03-26T04:13:32Z',
 '2023-04-16T03:14:32Z',
 '2023-04-30T13:17:33Z',
 '2023-05-03T14:20:40Z',
 '2023-05-04T01:37:10Z',
 '2023-05-14T14:02:25Z',
 '2023-05-21T14:34:23Z',
 '2023-05-28T13:34:58Z',
 '2023-06-11T14:42:37Z',
 '2023-06-24T02:38:44Z',
 '2023-07-02T09:41:26Z',
 '2023-07-09T12:29:20Z',
 '2023-07-20T12:53:52Z',
 '2023-07-23T10:45:22Z',
 '2023-07-30T05:03:29Z',
 '2023-08-06T13:00:28Z',
 '2023-08-16T10:26:23Z',
 '2023-08-20T08:22:18Z',
 '2023-09-04T14:23:54Z',
 '2023-09-10T01:47:23Z',
 '2023-09-17T11:40:14Z',
 '2023-09-24T14:02:05Z',
 '2023-10-06T02:45:23Z',
 '2023-10-10T13:09:41Z',
 '2023-10-15T09:45:18Z',
 '2023-10-17T01:17:18Z',
 '2023-10-29T09:43:56Z',
 '2023-11-19T05:04:26Z']

In [29]:
# Get the time series metrics (remediation_minutes) of the repository

response = requests.get(
    url=f"https://api.codeclimate.com/v1/repos/{repository_id}/metrics/technical_debt_ratio?filter[from]={release_timestamps[0]}&filter[to]={release_timestamps[-1]}",
    headers=default_headers,
)

response.json()["data"]

{'id': '655a1137a0367900010000ba',
 'type': 'metrics',
 'attributes': {'name': 'technical_debt_ratio',
  'points': [{'timestamp': 1677456000, 'value': None},
   {'timestamp': 1678060800, 'value': None},
   {'timestamp': 1678665600, 'value': None},
   {'timestamp': 1679270400, 'value': None},
   {'timestamp': 1679875200, 'value': None},
   {'timestamp': 1680480000, 'value': None},
   {'timestamp': 1681084800, 'value': None},
   {'timestamp': 1681689600, 'value': None},
   {'timestamp': 1682294400, 'value': None},
   {'timestamp': 1682899200, 'value': None},
   {'timestamp': 1683504000, 'value': None},
   {'timestamp': 1684108800, 'value': None},
   {'timestamp': 1684713600, 'value': None},
   {'timestamp': 1685318400, 'value': None},
   {'timestamp': 1685923200, 'value': None},
   {'timestamp': 1686528000, 'value': None},
   {'timestamp': 1687132800, 'value': None},
   {'timestamp': 1687737600, 'value': None},
   {'timestamp': 1688342400, 'value': None},
   {'timestamp': 1688947200, 'va